In [ ]:
import pynetlogo
import numpy as np
import matplotlib.pyplot as plt
import pygame
import json

In [ ]:
class NetLogoWrapper:
    def __init__(self, *, model_file, config_file, render=True) -> None:
        self.netlogo = pynetlogo.NetLogoLink(gui=False, jvm_path='/Applications/NetLogo 6.3.0/runtime/Contents/MacOS/libjli.dylib')
        self.netlogo.load_model(model_file)

        # Load and apply config
        with open(config_file, 'r') as f:
            self.config = json.load(f)

        self.resolution = 10
        self.max_x_cor = self.config['evaluation']['max_x_cor']
        self.max_y_cor = self.config['evaluation']['max_y_cor']
        self.netlogo.command(f'resize-world {-self.max_x_cor} {self.max_x_cor} {-self.max_y_cor} {self.max_y_cor}')
        self.netlogo.command(f'set num-cars {self.config["evaluation"]["num_cars"]}')
        self.netlogo.command(f'set num-garages {self.config["evaluation"]["num_garages"]}')
        self.netlogo.command(f'set demand-curve-intercept {self.config["evaluation"]["demand_curve_intercept"]}')
        self.netlogo.command(f'set lot-distribution-percentage {self.config["evaluation"]["lot_distribution_percentage"]}')
        self.netlogo.command(f'set target-start-occupancy {self.config["evaluation"]["target_start_occupancy"]}')

        if render:
            pygame.init()
            self.screen = pygame.display.set_mode((self.resolution * (2 * self.max_x_cor + 1), self.resolution * (2 * self.max_y_cor + 1)))
            print(f'size: {self.resolution * (2 * self.max_x_cor + 1)} x {self.resolution * (2 * self.max_y_cor + 1)}')
            pygame.display.set_caption('Restricted Parking')
            
        else:
            self.screen = None


    def setup(self):
        self.netlogo.command('setup')

    def step(self, number_of_steps=1):
        self.netlogo.repeat_command('go', number_of_steps)

    def report(self, variable):
        return self.netlogo.report(variable)

    def command(self, _command):
        self.netlogo.command(_command)

    def render_patches(self):
        self.screen.fill((0,0,0))

        for x, y, *color in self.netlogo.report('[(sentence pxcor pycor (ifelse-value (is-number? pcolor) [extract-rgb pcolor] [pcolor]))] of patches'):
            pygame.draw.rect(self.screen, color, pygame.Rect(*self.transform_coordinates(x, y), self.resolution, self.resolution))

        pygame.display.flip()
    
    def render(self):
        if not self.render:
            raise NotImplementedError
        else:
            self.screen.fill((0,0,0))
            pygame.display.flip()

    def close(self):
        pygame.quit()

    def transform_coordinates(self, x, y):
        return ((self.max_x_cor + x) * self.resolution, (self.max_y_cor - y) * self.resolution)

In [ ]:
class ParkingGovernance:
    def __init__(self, netlogo) -> None:
        self.netlogo = netlogo

    def act(self):
        pass

In [ ]:
wrapper = NetLogoWrapper(model_file='model.nlogo', config_file='model_config.json', render=False)
wrapper.setup()
# wrapper.render_patches()

In [ ]:
wrapper = NetLogoWrapper(model_file='model.nlogo', config_file='model_config.json', render=False)
wrapper.setup()
wrapper.command('ask blue-lot [set allowed? false]')

print(wrapper.report('[allowed?] of green-lot'))

number_of_measurements = 10

blue_occupancy = np.empty(number_of_measurements)
for t in range(number_of_measurements):
    wrapper.step(number_of_steps=10)
    blue_occupancy[t] = wrapper.report('count cars-on blue-lot / count blue-lot')

plt.plot(blue_occupancy)

: 